In [1]:
!pip install kagglehub kaggle

  Using cached kagglehub-0.3.12-py3-none-any.whl.metadata (38 kB)
Using cached kagglehub-0.3.12-py3-none-any.whl (67 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [kaggle]━━━━ 1/2 [kaggle]


In [ ]:
import kagglehub

kagglehub.dataset_download()